In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
from tqdm import tqdm
%matplotlib inline

# Camera Calibration

In [ ]:
cal_images = glob.glob('camera_cal/*.jpg')

objpoints = []  # 3D object points
imgpoints = []  # 2D image points
nx = 9
ny = 5
objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

for cal_img in tqdm(cal_images):
    img = cv2.imread(cal_img)
    #plt.imshow(img)
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [ ]:
img = cv2.imread(cal_images[0])
dst = cv2.undistort(img, mtx, dist, None, mtx)

plt.figure(figsize=(10,4))
plt.subplot(121)
plt.imshow(img[:,:,::-1])
plt.title('original')
plt.subplot(122)
plt.imshow(dst[:,:,::-1])
plt.title('undistorted')

# Undistort Test Images

In [ ]:
input_dir = 'test_images/'
output_dir = 'output_images/undistort/'

input_fnames = glob.glob(os.path.join(input_dir, '*.jpg'))

for fname in tqdm(input_fnames):
    img = cv2.imread(fname)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imwrite(os.path.join(output_dir, os.path.basename(fname)), dst)

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(121)
plt.imshow(img[:,:,::-1])
plt.title('original')
plt.subplot(122)
plt.imshow(dst[:,:,::-1])
plt.title('undistorted')

# Generate lane pixels

In [ ]:
class piplineParameters():
    def __init__(self):
        self.kernel_size=11 # for gaussian blur
        self.low_threshold = 30
        self.high_threshold = 100 # for canny edge detection
        # hough transformation parameters
        self.rho = 2 # distance resolution in pixels of the Hough grid
        self.theta = np.pi/180 # angular resolution in radians of the Hough grid
        self.threshold = 20 # minimum number of votes (intersections in Hough grid cell)
        self.min_line_len = 40 #minimum number of pixels making up a line
        self.max_line_gap = 40 # maximum gap in pixels between connectable line segments)

params = piplineParameters()

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=4, debug=False):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    if 0:  # draw all line segments
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    else:  # cluster line segments
        img_shape = img.shape
        # print(len(lines))
        left_x = []
        left_y = []
        left_w = []
        right_x = []
        right_y = []
        right_w = []
        min_y = img.shape[0]
        mid_x = img.shape[1] /2 
        max_x = img.shape[1]
        for line in lines:
            x1, y1, x2, y2 = line[0]
            s = float((y2-y1)/(x2-x1))  # slope                        
            b = float(y1*x2 - y2*x1) / (x2-x1)  # interscept as in y=s*x+b
            if s !=0:
                x_intercept = (img.shape[0]-b)/s
            else:
                x_intercept = np.inf
            if y1 < min_y:
                min_y = y1
            if y2 < min_y:
                min_y = y2
            if -5< s < -0.4 and 0.0 < x_intercept/max_x < 0.5 and x1 < mid_x*1.1 and x2 < mid_x*1.1:
                left_x += [x1, x2]
                left_y += [y1, y2]
                ww = (y2-y1)**2 + (x1-x2)**2
                left_w += [ww, ww]
                if debug:
                    cv2.line(img, (x1, y1), (x2, y2), [0,255,0], thickness)
            elif 5> s > 0.4 and 0.5 < x_intercept/max_x < 1.0 and x1 > mid_x*0.9 and x2 > mid_x*0.9:
                right_x += [x1, x2]
                right_y += [y1, y2]
                ww = (y2-y1)**2 + (x1-x2)**2
                right_w += [ww, ww]
                if debug:
                    cv2.line(img, (x1, y1), (x2, y2), [0,0,255], thickness)
            else: # drop outlier
                # print(s)
                # print(x_intercept/max_x)
                # print(x1,y1, x2,y2)
                if debug:
                    cv2.line(img, (x1, y1), (x2, y2), [255,0,255], 2)
        if len(left_y) >=2:
            left_lane = np.poly1d(np.polyfit(left_y, left_x, 1, w = left_w))
            
            yhat = left_lane(left_y)
            ybar = np.sum(left_x)/len(left_x)
            ssreg = np.sum((yhat-ybar)**2)
            sstot = np.sum((left_x - ybar)**2)
            gof = ssreg / sstot
            if gof < 0.7 and debug:
                print('left lane gof:', gof)
            elif gof >0.7:
                min_y = int(img_shape[0]*0.8)
                cv2.line(img, (int(left_lane(img_shape[0])), img_shape[0]), (int(left_lane(min_y)), min_y), color, thickness)
        if len(right_y) >=2:
            right_lane = np.poly1d(np.polyfit(right_y, right_x, 1, w = right_w))
            
            yhat = right_lane(right_y)
            ybar = np.sum(right_x)/len(right_x)
            ssreg = np.sum((yhat-ybar)**2)
            sstot = np.sum((right_x - ybar)**2)
            gof = ssreg / sstot
            if gof < 0.7 and debug:
                print('right lane gof:', gof)
            elif gof > 0.7:
                min_y = int(img_shape[0]*0.8)
                cv2.line(img, (int(right_lane(img_shape[0])), img_shape[0]), (int(right_lane(min_y)), min_y), color, thickness)
        

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def pipeline(img, visualize=True):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    
    img = np.copy(img)
    if visualize:
        plt.figure()
        plt.imshow(img[:,:,::-1])
    
    # normalize brightness
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv[:,:,2] = cv2.equalizeHist(np.uint8(hsv[:,:,2]))
    
    
    # split to 4 quandrant
    #midx = int(hls.shape[0]/2)
    #midy = int(hls.shape[1]/2)
    #hls[:midx,:midy,1] = cv2.equalizeHist(np.uint8(hls[:midx,:midy,1]))
    #hls[midx:,:midy,1] = cv2.equalizeHist(np.uint8(hls[midx:,:midy,1]))
    #hls[:midx,midy:,1] = cv2.equalizeHist(np.uint8(hls[:midx,midy:,1]))
    #hls[midx:,midy:,1] = cv2.equalizeHist(np.uint8(hls[midx:,midy:,1]))
    #hls[midx:,:,1] = cv2.equalizeHist(np.uint8(hls[midx:,:,1]))
    
    
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    if visualize:
        plt.figure()
        plt.imshow(img[:,:,::-1])

    
    sobel_ksize=3

    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS).astype(np.float)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float)

    # hist equalization
    if 0:
        for i in range(3):
            hsv[:,:,i] = clahe.apply(hsv[:,:,i].astype(np.uint16))
        for i in range(3):
            hls[:,:,i] = clahe.apply(hls[:,:,i].astype(np.uint16))
        gray = clahe.apply(gray.astype(np.uint16))
    
    #plt.figure()
    #plt.imshow(gray)
    
    # color thresholding
    color_binary = []
   
    # using gray
    cb = np.zeros_like(gray)
    cb_thresh = (200, 255)
    cb[(gray >= cb_thresh[0]) & (gray <= cb_thresh[1])] = 1
    color_binary.append(cb)
    
    # using s
    cb = np.zeros_like(gray)
    cb_thresh = (160, 255)
    cb[(hls[:,:,2] >= cb_thresh[0]) & (hls[:,:,2] <= cb_thresh[1])] = 1
    color_binary.append(cb)
    
    # using v
    cb = np.zeros_like(gray)
    cb_thresh = (210, 255)
    cb[(hsv[:,:,2] >= cb_thresh[0]) & (hsv[:,:,2] <= cb_thresh[1])] = 1
    color_binary.append(cb)
    
    color_binary = np.stack(color_binary, axis=2) *255
    
    if visualize:
        #print(color_binary.shape)
        plt.figure(figsize=(20,4))
        plt.subplot(141)
        plt.imshow(color_binary[:,:,0], cmap='gray')
        plt.subplot(142)
        plt.imshow(color_binary[:,:,1], cmap='gray')
        plt.subplot(143)
        plt.imshow(color_binary[:,:,2], cmap='gray')
    
    imshape=gray.shape
    vertices = np.array([[(imshape[1]*.15,imshape[0]*.95), (imshape[1]*0.48, imshape[0]*0.6), 
                        (imshape[1]*0.52,imshape[0]*0.6), (imshape[1]*.9,imshape[0]*.9)]], dtype=np.int32)
    
    
    color_binary = region_of_interest(color_binary, vertices)
    
    if visualize:
        plt.figure(figsize=(20,4))
        plt.subplot(141)
        plt.imshow(color_binary[:,:,0], cmap='gray')
        plt.subplot(142)
        plt.imshow(color_binary[:,:,1], cmap='gray')
        plt.subplot(143)
        plt.imshow(color_binary[:,:,2], cmap='gray')
    

    sbx_binary = []
    
    # using s
    sbx = np.zeros_like(gray)
    sobelx = cv2.Sobel(hls[:,:,2], cv2.CV_64F, 1, 0, ksize=sobel_ksize) 
    sobelx = np.absolute(sobelx)
    sobelx = np.uint8(255*sobelx/np.max(sobelx))   
    sx_thresh=(40, 255)
    sbx[(sobelx >= sx_thresh[0]) & (sobelx <= sx_thresh[1])] = 1
    sbx_binary.append(sbx)
    
    # using l
    sbx = np.zeros_like(gray)
    sobelx = cv2.Sobel(hls[:,:,1], cv2.CV_64F, 1, 0, ksize=sobel_ksize) 
    sobelx = np.absolute(sobelx)
    sobelx = np.uint8(255*sobelx/np.max(sobelx))   
    sx_thresh=(60, 255)
    sbx[(sobelx >= sx_thresh[0]) & (sobelx <= sx_thresh[1])] = 1
    sbx_binary.append(sbx)
    
    # using hough
    #img_blur = gaussian_blur(hsv[:,:,2], params.kernel_size)
    img_blur = gaussian_blur(color_binary[:,:,2], params.kernel_size)
    img_canny = canny(np.uint8(img_blur), params.low_threshold, params.high_threshold)
    masked_canny = region_of_interest(img_canny, vertices)
    line_img = hough_lines(masked_canny, params.rho, params.theta, params.threshold, params.min_line_len, params.max_line_gap)
    

    sbx_binary.append(line_img)
    
    sbx_binary = np.stack(sbx_binary, axis=2)

    sbx_binary = region_of_interest(sbx_binary, vertices)
    
    if visualize:
        plt.figure(figsize=(20,4))
        plt.subplot(141)
        plt.imshow(sbx_binary[:,:,0]*255, cmap='gray')
        plt.subplot(142)
        plt.imshow(sbx_binary[:,:,1], cmap='gray')
        plt.subplot(143)
        plt.imshow(sbx_binary[:,:,2], cmap='gray')
    
    for i in range(3):
        #print(color_binary[:,:,i].mean())
        #print(sbx_binary[:,:,i].mean())
        if color_binary[:,:,i].mean() > 5:
            color_binary[:,:,i]= np.zeros_like(color_binary[:,:,i])
#         if sbx_binary[:,:,i].mean() > 5:
#             sbx_binary[:,:,i]= np.zeros_like(sbx_binary[:,:,i])
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(gray)
    sum_matrix = sbx_binary.sum(axis=2) + color_binary.sum(axis=2)
    combined_binary[sum_matrix>0] = 255
    
    if visualize:
        plt.figure()
        plt.imshow(combined_binary, cmap='gray')
        plt.title('combined')
    
    return combined_binary

In [ ]:
img = cv2.imread(input_fnames[-1])
dst = pipeline(img)

In [ ]:
input_dir = 'output_images/undistort/'
output_dir = 'output_images/binary/'
input_fnames = glob.glob(os.path.join(input_dir, '*.jpg'))

for fname in tqdm(input_fnames):
    img = cv2.imread(fname)
    dst = pipeline(img, visualize=True)
    cv2.imwrite(os.path.join(output_dir, os.path.basename(fname)), dst)

#result = pipeline(cv2.imread(input_fnames[1]), visualize=True)
#result = pipeline(cv2.imread(input_fnames[2]), visualize=True)

# Perspective Transform

In [ ]:
imshape = img.shape
pts = np.array([[imshape[1]*0.16, imshape[0]*1.0], [imshape[1]*0.46, imshape[0]*0.63],
                [imshape[1]*0.54, imshape[0]*0.63], [imshape[1]*0.87, imshape[0]*1.0]], dtype=np.int32)

dst = np.array([[imshape[1]*0.2, imshape[0]*0.95], [imshape[1]*0.2, imshape[0]*0.05],
                [imshape[1]*0.8, imshape[0]*0.05], [imshape[1]*0.8, imshape[0]*0.95]], dtype=np.int32)

pts = np.float32(pts)
dst = np.float32(dst)
M = cv2.getPerspectiveTransform(pts, dst)
Minv = cv2.getPerspectiveTransform(dst, pts)

input_dir = 'output_images/undistort/'
output_dir = 'output_images/warped/'
input_fnames = glob.glob(os.path.join(input_dir, '*.jpg'))

for fname in tqdm(input_fnames):    
    img = cv2.imread(fname)
    imshape = img.shape

    roi = np.copy(img)
    _ = cv2.polylines(roi, [np.int32(pts)], True, (0,0,255),3)
    cv2.imwrite(os.path.join(output_dir, os.path.basename(fname.replace('.jpg', '_original.jpg'))), roi)
    
    #plt.imshow(roi[:,:,::-1])
    

    warped = cv2.warpPerspective(img, M, imshape[:-1][::-1], flags=cv2.INTER_LINEAR)    
    warped_roi = np.copy(warped)
    _ = cv2.polylines(warped_roi, [np.int32(dst)], True, (0,0,255),3)
    cv2.imwrite(os.path.join(output_dir, os.path.basename(fname.replace('.jpg', '_warped.jpg'))), warped_roi)
    
    #plt.imshow(warped[:,:,::-1])

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(121)
plt.imshow(roi[:,:,::-1])
plt.title('undistorted')
plt.subplot(122)
plt.imshow(warped_roi[:,:,::-1])
plt.title('warped')

In [ ]:
input_dir = 'output_images/binary/'
output_dir = 'output_images/binary_warped/'
input_fnames = glob.glob(os.path.join(input_dir, '*.jpg'))

for fname in tqdm(input_fnames):    
    img = cv2.imread(fname)
    imshape = img.shape
    warped = cv2.warpPerspective(img, M, imshape[:-1][::-1], flags=cv2.INTER_LINEAR)    
    cv2.imwrite(os.path.join(output_dir, os.path.basename(fname)), warped)
    #plt.imshow(warped[:,:,::-1])

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(121)
plt.imshow(img[:,:,::-1])
plt.title('binary')
plt.subplot(122)
plt.imshow(warped[:,:,::-1])
plt.title('binary warped')

# Find lanes

In [ ]:
input_dir = 'output_images/binary_warped/'
output_dir = 'output_images/lane_labeled/'
output_dir2 = 'output_images/lane_labeled_unwarped/'
input_fnames = glob.glob(os.path.join(input_dir, '*.jpg'))

In [ ]:
for fname in tqdm(input_fnames):
    img = cv2.imread(fname)
    binary_warped = img
    
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]*.5):,:, 0], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = img
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 15
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 70
    # Set minimum number of pixels found to recenter window
    minpix = 40
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 5) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 5) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    plt.figure()
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    lane_center = (left_fitx + right_fitx )/2
    plt.plot(lane_center, ploty, color='red')
    # plt.xlim(0, 1280)
    # plt.ylim(720, 0)
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    off_center_to_the_left = (lane_center[-1] - out_img.shape[1]/2) * xm_per_pix
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    y_eval = ploty.max()
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    
    info_str = 'off center to the left by {:.2f}m\n left radius: {:.2f}m, right radius: {:.2f}m'.format(off_center_to_the_left,
                                                                                           left_curverad, 
                                                                                            right_curverad)
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.title(info_str)
    plt.savefig(os.path.join(output_dir, os.path.basename(fname)))
    #plt.close()
    
    # Create an image to draw the lines on
    color_warp = np.zeros_like(img).astype(np.uint8)

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image

    undistort = cv2.imread(os.path.join('output_images/undistort/', os.path.basename(fname)))
    result = cv2.addWeighted(undistort, 1, newwarp, 0.3, 0)
    cv2.imwrite(os.path.join(output_dir2, os.path.basename(fname)), result)

In [ ]:
# Assume you now have a new warped binary image 
# from the next frame of video (also called "binary_warped")
# It's now much easier to find line pixels!
nonzero = binary_warped[:,:,0].nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

# Again, extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds]
# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)
# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

In [ ]:
# Create an image to draw on and an image to show the selection window
out_img = img
window_img = np.zeros_like(out_img)
# Color in left and right line pixels
out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))
right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

# Processing Videos

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        
        self.last_N_fit = []
        self.keep_N_fit = 5
        
        self.off_center_to_the_left = 0
    
leftlane = Line()
rightlane = Line()

In [ ]:
imshape = img.shape
pts = np.array([[imshape[1]*0.16, imshape[0]*1.0], [imshape[1]*0.46, imshape[0]*0.63],
                [imshape[1]*0.54, imshape[0]*0.63], [imshape[1]*0.87, imshape[0]*1.0]], dtype=np.int32)

dst = np.array([[imshape[1]*0.2, imshape[0]*0.95], [imshape[1]*0.2, imshape[0]*0.05],
                [imshape[1]*0.8, imshape[0]*0.05], [imshape[1]*0.8, imshape[0]*0.95]], dtype=np.int32)

pts = np.float32(pts)
dst = np.float32(dst)
M = cv2.getPerspectiveTransform(pts, dst)
Minv = cv2.getPerspectiveTransform(dst, pts)

def process_image(img):        
    undistort = cv2.undistort(img, mtx, dist, None, mtx)
    binary = pipeline(undistort)
    binary_warped = cv2.warpPerspective(binary, M, imshape[:-1][::-1], flags=cv2.INTER_LINEAR)        
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))


    nonzero = binary_warped[:,:].nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Set the width of the windows +/- margin
    margin = 70
    # Set minimum number of pixels found to recenter window
    minpix = 40

    
    if leftlane.detected and rightlane.detected:
        left_fit = leftlane.best_fit
        right_fit = rightlane.best_fit
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  
    else:  
        histogram = np.sum(binary_warped[int(binary_warped.shape[0]*.5):,:], axis=0)
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Choose the number of sliding windows
        nwindows = 15
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)

        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base

        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 5) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 5) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
        
        
        # prefer to use exiting fit, overwrting the above
        if leftlane.detected:
            left_fit = leftlane.best_fit
            left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        if rightlane.detected:
            right_fit = rightlane.best_fit
            right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  


    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    if leftx.size >0:
        left_fit = np.polyfit(lefty, leftx, 2)
    else:
        if leftlane.detected:
            left_fit = leftlane.best_fit
        else:
            left_fit = None
    
    if rightx.size >0:
        right_fit = np.polyfit(righty, rightx, 2)
    else:
        if rightlane.detected:
            right_fit = rightlane.best_fit
        else:
            right_fit = None
    
    if right_fit is None and left_fit is None:
        assert 0
    else:
        if right_fit is None:
            right_fit = left_fit.copy()
            right_fit[2] += 700
        if left_fit is None:
            left_fit = right_fit.copy()
            left_fit[2] -= 700
    
    
    # record fit in Line class
    leftlane.detected = True
    rightlane.detected = True
    leftlane.current_fit = left_fit
    rightlane.current_fit = right_fit
    leftlane.last_N_fit.append(left_fit)
    rightlane.last_N_fit.append(right_fit)
    if len(leftlane.last_N_fit) > leftlane.keep_N_fit:
        leftlane.last_N_fit.pop(0)
    if len(rightlane.last_N_fit) > rightlane.keep_N_fit:
        rightlane.last_N_fit.pop(0)
    
    leftlane.best_fit = np.mean(leftlane.last_N_fit, axis=0)
    rightlane.best_fit = np.mean(rightlane.last_N_fit, axis=0)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    #plt.figure()
    #plt.imshow(out_img)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    lane_center = (left_fitx + right_fitx )/2
    #plt.plot(lane_center, ploty, color='red')
    # plt.xlim(0, 1280)
    # plt.ylim(720, 0)
    
    if leftx.size >0 and rightx.size>0:
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension

        off_center_to_the_left = (lane_center[-1] - out_img.shape[1]/2) * xm_per_pix

        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
        # Calculate the new radii of curvature
        y_eval = ploty.max()
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        # Now our radius of curvature is in meters
        
        leftlane.radius_of_curvature = left_curverad
        rightlane.radius_of_curvature = right_curverad
        leftlane.off_center_to_the_left = off_center_to_the_left
    
    info_str = 'off center to the left by {:.2f}m\n left radius: {:.2f}m, right radius: {:.2f}m'.format(leftlane.off_center_to_the_left,
                                                                                                        leftlane.radius_of_curvature,
                                                                                                       rightlane.radius_of_curvature)
    
    
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    #plt.title(info_str)
    #plt.savefig(os.path.join(output_dir, os.path.basename(fname)))
    #plt.close()
    
    # Create an image to draw the lines on
    color_warp = np.zeros_like(img).astype(np.uint8)

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image

    #undistort = cv2.imread(os.path.join('output_images/undistort/', os.path.basename(fname)))
    result = cv2.addWeighted(undistort, 1, newwarp, 0.3, 0)
    #cv2.imwrite(os.path.join(output_dir2, os.path.basename(fname)), result)

    return result

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
video_output = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
## clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
output_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time output_clip.write_videofile(video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))

In [ ]:
video_input = 'challenge_video.mp4'
video_output = 'challenge_video_output.mp4'
clip1 = VideoFileClip(video_input)

if 0:
    for t in np.linspace(0, clip1.duration, 10):
        clip1.save_frame(os.path.join('test_images', video_input.replace('.mp4', '_t={:.2f}.jpg'.format(t))), t=t)
else:
    output_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
    %time output_clip.write_videofile(video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))

In [ ]:
video_input = 'harder_challenge_video.mp4'
video_output = 'harder_challenge_video_output.mp4'
clip1 = VideoFileClip(video_input)
if 0:
    for t in np.linspace(0, clip1.duration, 10):
        clip1.save_frame(os.path.join('test_images', video_input.replace('.mp4', '_t={:.2f}.jpg'.format(t))), t=t)
else:
    output_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
    %time output_clip.write_videofile(video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))